# Predicting octane numbers from NIR spectra

In this example, we will predict [octane numbers](https://en.wikipedia.org/wiki/Octane_rating) from measured NIR spectra. Specifically, we will compare a least squares model to a partial least squares (PLS) model and we will
use cross-validation to check the performance of the models.

## Reading the raw data

The raw data can be found in the file [gasoline.csv](./gasoline.csv). We load it with pandas and
extract the octane number, the spectra, and the wavelengths for the spectra.

In [ ]:
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

sns.set_theme(style="ticks", context="notebook", palette="colorblind")
%matplotlib notebook

np.random.seed(2025)  # Set random state to get the same results for each run

In [ ]:
# Load the data set:
data = pd.read_csv("gasoline.csv")
# Each row contains a measured spectrum and a corresponding octane number:
# 1. extract the octane numbers:
yvars = ["octane"]
octane = data[yvars].to_numpy()
# 2. extract the spectra (intensities):
xvars = [i for i in data.columns if i not in yvars]
spectra = data[xvars].to_numpy()
# 3. get the wavelengths:
wavelengths = np.array([int(i.split()[0].split(".")[1]) for i in xvars])
print(f"Number of wavelengths measured: {len(xvars)}")
data.describe()

In [ ]:
# Let us visualize the spectra, just to see what we have to work with:
# We add some color, so we can color the spectra according to
# the octane numbers:
norm = mpl.colors.Normalize(vmin=octane.min(), vmax=octane.max())
cmap = mpl.cm.ScalarMappable(norm=norm, cmap="coolwarm")

fig, axi = plt.subplots(constrained_layout=True)
axi.set_title("NIR spectra")
for i, speci in enumerate(spectra):
    axi.plot(wavelengths, speci, color=cmap.to_rgba(octane[i]))
axi.set(xlabel="Wavelength (nm)", ylabel="Absorbance")
# axi.set_facecolor("0.975")
fig.colorbar(cmap, ax=axi, label="Octane numbers")
sns.despine(fig=fig)

The spectra show variations in intensities related to octane number (for instance, around 1200-1220 nm where the samples with lower octane number have higher absorbances compared to samples with higher octane numbers). We proceed to explore the feasibility of constructing regression models that accurately predict octane values from these spectral data.

## Comparing least squares, partial least squares, and LASSO regression with cross-validation for predicting octane numbers

### Create testing and training sets:

In [ ]:
from sklearn.model_selection import (
    GridSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.preprocessing import StandardScaler

# Create training and test sets:
X_train, X_test, Y_train, Y_test = train_test_split(
    spectra, octane, test_size=0.33, random_state=2025
)

# To scale X:
#scaler_x = StandardScaler().fit(X_train)
#X_train = scaler_x.transform(X_train)
#X_test = scaler_x.transform(X_test)

# To scale y:
scaler_y = StandardScaler().fit(Y_train)
Y_train = scaler_y.transform(Y_train)
Y_test = scaler_y.transform(Y_test)

### Create a least squares model

In [ ]:
from sklearn.linear_model import LinearRegression

leastsquares = LinearRegression(fit_intercept=False)
leastsquares.fit(X_train, Y_train)
B_MLR = leastsquares.coef_[0, :]

### Create a partial least squares model

In [ ]:
# Create a PLS model and use cross-validation to find
# the best number of pls components to use:
from sklearn.cross_decomposition import PLSRegression

parameters = {
    "n_components": range(1, 11),
}

grid = GridSearchCV(
    PLSRegression(scale=False),
    parameters,
    scoring="neg_mean_squared_error",
    refit=True,
)

grid.fit(X_train, Y_train)
print(grid.best_params_)
pls_model_optimized = grid.best_estimator_
B_PLS = pls_model_optimized.coef_[0, :]

In [ ]:
fig, axi = plt.subplots(constrained_layout=True)
score = grid.cv_results_["mean_test_score"]
score_std = grid.cv_results_["std_test_score"]
axi.errorbar(
    parameters["n_components"],
    score,
    yerr=score_std,
    marker="o",
)
axi.set_xticks(parameters["n_components"])
axi.set(
    xlabel="PLS components", ylabel="score", title="Results from grid search"
)
sns.despine(fig=fig)

In [ ]:
# pls_model_optimized = PLSRegression(n_components=3).fit(X_train, Y_train)

### Create a LASSO model

In [ ]:
from sklearn.linear_model import Lasso

parameters_lasso = {"alpha": np.logspace(-3, 2, 10)}

grid_lasso = GridSearchCV(
    Lasso(fit_intercept=False, max_iter=10000, random_state=2025),
    parameters_lasso,
    scoring="neg_mean_squared_error",
    refit=True,
)
grid_lasso.fit(X_train, Y_train)
print(grid_lasso.best_params_)
lasso_model_optimized = grid_lasso.best_estimator_
B_lasso = lasso_model_optimized.coef_

### Check performance with cross-validation

In [ ]:
fig, axes = plt.subplots(
    constrained_layout=True, ncols=3, sharex=True, sharey=True, figsize=(9, 3)
)

axes[0].set_title("Partial least squares", loc="left")
axes[1].set_title("Least squares", loc="left")
axes[2].set_title("Lasso", loc="left")

models = [pls_model_optimized, leastsquares, lasso_model_optimized]

for i, model in enumerate(models):
    cv_results = cross_validate(
        model,
        X_train,
        Y_train,
        scoring=("r2", "neg_mean_squared_error"),
        return_train_score=True,
        cv=5,
    )
    axes[i].plot(cv_results["train_r2"], label="Training", marker="o")
    axes[i].plot(cv_results["test_r2"], label="Test", marker="o")
    axes[i].set_xlabel("CV no.")
axes[0].set_ylabel("R²")
axes[0].legend()
sns.despine(fig=fig)

In [ ]:
from sklearn.metrics import root_mean_squared_error, r2_score


def get_scores(y, y_hat):
    r2 = r2_score(y, y_hat)
    rmse = root_mean_squared_error(y, y_hat)
    return r2, rmse


def add_scores(model, X_train, y_train, X_test, y_test, ax):
    """Plot y vs y_hat for test and training."""
    y_hat_train = model.predict(X_train)
    y_hat_test = model.predict(X_test)

    r2_train, rmsec = get_scores(y_train, y_hat_train)
    r2_test, rmsep = get_scores(y_test, y_hat_test)

    txt_train = f"R²(train) = {r2_train:.2f}\nRMSEC = {rmsec:.2f}"
    txt_test = f"R²(test) = {r2_test:.2f}\nRMSEP = {rmsep:.2f}"

    ax.scatter(Y_train, y_hat_train, label=txt_train)
    ax.scatter(Y_test, y_hat_test, label=txt_test)
    ax.legend(fontsize="small")

In [ ]:
fig, axes = plt.subplots(
    constrained_layout=True,
    ncols=3,
    sharex=True,
    sharey=True,
    figsize=(9, 3),
)

axes[0].set_title("Partial least squares", loc="left")
axes[1].set_title("Least squares", loc="left")
axes[2].set_title("Lasso", loc="left")

models = [pls_model_optimized, leastsquares, lasso_model_optimized]

for i, model in enumerate(models):
    axes[i].set_aspect("equal")
    add_scores(model, X_train, Y_train, X_test, Y_test, axes[i])
    axes[i].set_xlabel("y")
axes[0].set_ylabel("ŷ")
sns.despine(fig=fig)

In [ ]:
fig, axes = plt.subplots(
    constrained_layout=True,
    nrows=3,
    sharex=True,
    figsize=(9,6)
)

axes[0].set_title("Partial least squares", loc="left")
axes[1].set_title("Least squares", loc="left")
axes[2].set_title("Lasso", loc="left")

coeffs = [B_PLS, B_MLR, B_lasso]

for i, coef in enumerate(coeffs):
    axes[i].plot(wavelengths, coef)
    axes[i].axhline(y=0.0, ls=":", color="k")
    axes[i].set_ylabel("Regression\ncoefficient")
axes[2].set_xlabel("Wavelength (nm)")
sns.despine(fig=fig)

In [ ]:
norm = mpl.colors.Normalize(vmin=octane.min(), vmax=octane.max())
cmap = mpl.cm.ScalarMappable(norm=norm, cmap="coolwarm")
fig, axi = plt.subplots(constrained_layout=True)
axi.set_title("NIR spectra")
for i, speci in enumerate(spectra):
    axi.plot(wavelengths, speci, color=cmap.to_rgba(octane[i]))

# axi.plot(wavelengths, B_lasso)
axi.set(xlabel="Wavelength (nm)", ylabel="Absorbance")
# axi.set_facecolor("0.975")
fig.colorbar(cmap, ax=axi, label="Octane numbers")
sns.despine(fig=fig)